In [ ]:
!pip install reportlab

In [ ]:
def validar_cpf(cpf):
    # Remove caracteres especiais do CPF, como pontos e traços
    cpf = ''.join(filter(str.isdigit, cpf))

    # Verifica se o CPF tem 11 dígitos
    if len(cpf) != 11:
        return False

    # Verifica se todos os dígitos são iguais (exemplo: 111.111.111-11)
    if cpf == cpf[0] * 11:
        return False

    # Calcula o primeiro dígito verificador
    soma = sum(int(cpf[i]) * (10 - i) for i in range(9))
    primeiro_digito = (soma * 10) % 11
    if primeiro_digito == 10:
        primeiro_digito = 0

    # Verifica o primeiro dígito
    if primeiro_digito != int(cpf[9]):
        return False

    # Calcula o segundo dígito verificador
    soma = sum(int(cpf[i]) * (11 - i) for i in range(10))
    segundo_digito = (soma * 10) % 11
    if segundo_digito == 10:
        segundo_digito = 0

    # Verifica o segundo dígito
    return segundo_digito == int(cpf[10])


In [ ]:
import re
import json

class ClienteManager:
    def __init__(self):
        self.clientes = []
        self.carregar_dados()

    def validar_email(self, email):
        # Expressão regular para validar email
        padrao = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
        return re.match(padrao, email) is not None

    def validar_cpf(self, cpf):
        # Validar o CPF
        return validar_cpf(cpf) 

    def cadastrar_cliente(self, nome, cpf, email, endereco, telefone):
        if not self.validar_cpf(cpf):
            raise ValueError("CPF inválido")
        if not self.validar_email(email):
            raise ValueError("Email inválido")
        
        cliente = {
            "nome": nome,
            "cpf": cpf,
            "email": email,
            "endereco": endereco,
            "telefone": telefone
        }
        self.clientes.append(cliente)
        self.salvar_dados()

    def excluir_cliente(self, cpf, motivo):
        for cliente in self.clientes:
            if cliente["cpf"] == cpf:
                self.clientes.remove(cliente)
                self.salvar_dados()
                return cliente
        raise ValueError("Cliente não encontrado")

    def salvar_dados(self):
        with open('clientes.json', 'w') as file:
            json.dump(self.clientes, file)

    def carregar_dados(self):
        try:
            with open('clientes.json', 'r') as file:
                self.clientes = json.load(file)
        except FileNotFoundError:
            self.clientes = []

In [ ]:
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import datetime

def gerar_pdf_cadastro(cliente):
    c = canvas.Canvas(f"{cliente['nome']}_cadastro.pdf", pagesize=letter)
    c.drawString(100, 750, f"Nome: {cliente['nome']}")
    c.drawString(100, 730, f"CPF: {cliente['cpf']}")
    c.drawString(100, 710, f"Data de Emissão: {datetime.now().strftime('%d/%m/%Y')}")
    c.save()

def gerar_pdf_exclusao(cliente, motivo):
    c = canvas.Canvas(f"{cliente['nome']}_exclusao.pdf", pagesize=letter)
    c.drawString(100, 750, f"Nome: {cliente['nome']}")
    c.drawString(100, 730, f"CPF: {cliente['cpf']}")
    c.drawString(100, 710, f"Data de Cancelamento: {datetime.now().strftime('%d/%m/%Y')}")
    c.drawString(100, 690, f"Motivo: {motivo}")
    c.save()


In [ ]:
def menu():
    print("1. Cadastrar Cliente")
    print("2. Excluir Cliente")
    print("3. Sair")
    return input("Escolha uma opção: ")

def main():
    manager = ClienteManager()

    while True:
        opcao = menu()

        if opcao == "1":
            nome = input("Nome: ")
            cpf = input("CPF: ")
            email = input("Email: ")
            endereco = input("Endereço: ")
            telefone = input("Telefone: ")

            try:
                manager.cadastrar_cliente(nome, cpf, email, endereco, telefone)
                gerar_pdf_cadastro({
                    "nome": nome,
                    "cpf": cpf,
                    "email": email,
                    "endereco": endereco,
                    "telefone": telefone
                })
                print("Cliente cadastrado com sucesso!")
            except ValueError as e:
                print(e)

        elif opcao == "2":
            cpf = input("CPF do cliente a ser excluído: ")
            motivo = input("Motivo da exclusão: ")

            try:
                cliente = manager.excluir_cliente(cpf, motivo)
                gerar_pdf_exclusao(cliente, motivo)
                print("Cliente excluído com sucesso!")
            except ValueError as e:
                print(e)

        elif opcao == "3":
            break

        else:
            print("Opção inválida.")


In [ ]:
main()